# Naive Bayes

In [4]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("naive").getOrCreate()

24/04/03 15:29:43 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlo1)
24/04/03 15:29:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 15:29:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
iris = spark.read.csv("../0_data/iris.csv", header=True, inferSchema=True, sep=",")
print(iris.count())
iris.show(5)

150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



In [7]:
RFormula = RFormula(formula="class ~ .", featuresCol="features", labelCol="label")
iris_rf = RFormula.fit(iris).transform(iris)
iris_rf.select("features", "label").show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
+-----------------+-----+
only showing top 5 rows



In [8]:
iris_train, iris_test = iris_rf.randomSplit([0.7, 0.3])
print(iris_train.count(), iris_test.count())

103 47


In [9]:
naive = NaiveBayes(smoothing=1.0,
                   modelType="multinomial",
                   featuresCol="features",
                   labelCol="label")
model = naive.fit(iris_train)

In [10]:
predictions = model.transform(iris_test)
predictions.select("label", "prediction", "probability", "rawPrediction").show(15, truncate=False)

+-----+----------+------------------------------------------------------------+-------------------------------------------------------------+
|label|prediction|probability                                                 |rawPrediction                                                |
+-----+----------+------------------------------------------------------------+-------------------------------------------------------------+
|0.0  |0.0       |[0.7197107224453221,0.17646910077458292,0.10382017678009492]|[-9.947959820667759,-11.353663382948934,-11.884148844410095] |
|0.0  |0.0       |[0.6342753041788916,0.22536296126185226,0.14036173455925618]|[-10.84674529967313,-11.881516129957475,-12.355005485008736] |
|0.0  |0.0       |[0.6826816127675961,0.19951567049448424,0.11780271673791956]|[-11.31624041333696,-12.546376222201467,-13.073257671048225] |
|0.0  |0.0       |[0.7852199262705716,0.13939639858296798,0.07538367514646048]|[-10.982729089217084,-12.711371265752664,-13.326102187037655]|
|0.0  

In [11]:
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = performance.evaluate(predictions)
print(f"accuracy: {accuracy:.2f}")

accuracy: 0.87
